In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
#In console run following:
#pip install scikit-image
#pip install graphviz
#pip install dask-image
#pip install imageio[pyav]
import dask
import matplotlib
import numpy
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import dask.array as da
import re
import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
labels = []
include50labels = ['Bird','Black','Car','Dog','Fall','Father','Good Morning', 'Red','Summer','White','loud','quiet','happy','long','short','big large', 'small little','hot', 'new','dry','good','Cow','Hat', 'T-Shirt', 'Shoes', 'Monday', 'Year', 'Time', 'Fan', 'Cell phone', 'Hello', 'Thank you', 'Window', 'Pen', 'Paint', 'Teacher', 'Priest', 'train ticket', 'Brother', 'Boy', ' Girl', ' Bank', 'I', 'it','you', 'Election', 'Death', 'Court', 'House', 'Store or Shop']


In [12]:
def return_label(dirname):
    return re.sub('\d+. ','',dirname)
datasets = ['include-50','include-50-2']
for dataset in datasets:
    os.chdir("/kaggle/input/{dataset_name}".format(dataset_name = dataset))
    root_dir_list = os.listdir()
    for root_dir in root_dir_list:
        os.chdir(os.path.join(os.getcwd(),root_dir))
        dir_list = os.listdir()
        for dir in dir_list:
            os.chdir(os.path.join(os.getcwd(),dir))
            n = len(os.listdir())
            label = return_label(dir)
            for i in range(n):
                labels.append(label)
            os.chdir('../')
        os.chdir('../')

In [13]:
extra_labels= { 'Bird':4,'Bank':1, 'Store or Shop':1,'Court':2}
def insert_extra_labels(labels, extra_labels):
    for extra_label in extra_labels:
        index = labels.index(extra_label)
        for i in range(extra_labels[extra_label]):
            labels.insert(index,extra_label)
    return labels

In [14]:
labels = insert_extra_labels(labels,extra_labels)

In [15]:
len(labels)

1146

In [ ]:
video_frames = []
for dataset in datasets:
    for root,_,filenames in os.walk("/kaggle/input/{dataset_name}".format(dataset_name = dataset)):
        for filename in filenames:
            #print(os.path.join(root,filename))
            file_path = os.path.join(root,filename)
            x = dask_image.imread.imread(file_path)
            video_frames.append(x)

In [1]:
len(video_frames)

NameError: name 'video_frames' is not defined

In [11]:
!git clone https://github.com/Hzzone/pytorch-openpose
%cd pytorch-openpose
import matplotlib.pyplot as plt
import copy
import numpy as np
from src import model
from src import util
from src.body import Body
from src.hand import Hand

!cp /kaggle/input/openpose-pretrained-models/body_pose_model.pth model
!cp /kaggle/input/openpose-pretrained-models/hand_pose_model.pth model

body_estimation = Body('model/body_pose_model.pth')
hand_estimation = Hand('model/hand_pose_model.pth')

Cloning into 'pytorch-openpose'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 154 (delta 69), reused 152 (delta 67), pack-reused 0
Receiving objects: 100% (154/154), 20.18 MiB | 25.58 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/kaggle/working/pytorch-openpose


In [61]:
len(video_frames[0])

56

In [62]:
video_frames[0]

,Array,Chunk
Bytes,332.23 MiB,5.93 MiB
Shape,"(56, 1080, 1920, 3)","(1, 1080, 1920, 3)"
Count,168 Tasks,56 Chunks
Type,uint8,numpy.ndarray


In [191]:
def get_all_hand_peaks(hands_list,oriImg):
    all_hand_peaks = []
    for x, y, w, is_left in hands_list:
        peaks = hand_estimation(oriImg[y:y+w, x:x+w, :]) #pretrained model, last layer: convolution_param-> num_output: 22, 21 Keypoints
        #peaks is coordinates wrt x,y top left.
        #adjust coordinates of keypoints
        peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], peaks[:, 0]+x)
        peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y) 
        all_hand_peaks.append(peaks)
    return all_hand_peaks

def generate_hand_keypoints_for_one_video(video):
    vdo_keypoints = []
    Num_of_frames = len(video)
    for f in range(Num_of_frames):
        try:
            oriImg = np.array(video[f])
        except:
            continue
        if(oriImg is None):
            continue
        candidate, subset = body_estimation(oriImg)
        canvas = copy.deepcopy(oriImg)
        hands_list = util.handDetect(candidate, subset, oriImg)
        all_hand_peaks = get_all_hand_peaks(hands_list,oriImg)
        vdo_keypoints.append(all_hand_peaks)
    return vdo_keypoints




In [189]:
vdo_keypoints_list = []
def generate_hand_keypoints_for_multiple_video(video_list,start_range, end_range):
    #(start_range, last_range], including start and excluding last.
    for i in range(start_range, end_range):
        print(i)
        vdo_keypoints_list.append(generate_hand_keypoints_for_one_video(video_list[i]))

In [190]:
len(vdo_keypoints_list)

0

In [ ]:
vdo_keypoints_list.append( generate_hand_keypoints_for_multiple_video(video_frames, 20,40) )

20
21
22
23
24
25
26
27
28
29


In [174]:
len(vdo_keypoints_list)
#num_of_vdosxnum_of_framesx2_handsx21_keypointsx2_cordinate(x,y)


10

In [134]:
#last element of the list if none then pop it using this, be careful to run this only once or u can loose our keypoints and ess up the range
#vdo_keypoints_list.pop()
print('Last element popped')

In [178]:
#if vdo_keypoints contains keypoints from ur last run, please make sure u select appropriate range
df = pd.DataFrame({'Label':labels[10:20],'Keypoints':vdo_keypoints_list})

In [183]:
df

,Label,Keypoints
0,hot,"[[[[0 0], [0 0], [0 0], [0 0], [0 0], [0 0], [..."
1,hot,"[[[[0 0], [0 0], [0 0], [0 0], [0 0], [0 0], [..."
2,hot,"[[[[0 0], [1084 772], [1092 791], [1088 809..."
3,hot,"[[[[0 0], [0 0], [0 0], [1137 889], [1130 90..."
4,hot,"[[[[0 0], [0 0], [0 0], [0 0], [0 0], [0 0], [..."
5,hot,"[[[[0 0], [0 0], [0 0], [0 0], [0 0], [0 0], [..."
6,hot,"[[[[1123 841], [0 0], [0 0], [0 0], [0 0], [0..."
7,hot,"[[[[0 0], [0 0], [0 0], [0 0], [0 0], [0 0], [..."
8,hot,"[[[[0 0], [0 0], [0 0], [0 0], [0 0], [0 0], [..."
9,hot,"[[[[0 0], [0 0], [0 0], [0 0], [0 0], [0 0], [..."


In [182]:
%cd /kaggle/working


/kaggle/working


In [185]:
df.to_csv('Video_keypoints_two.csv')

In [186]:
ls

Video_keypoints.csv      __notebook_source__.ipynb  state.db
Video_keypoints_two.csv  pytorch-openpose/


In [52]:
from IPython.display import FileLink
FileLink(r'Video_keypoints_.csv')

/kaggle/working/Video_keypoints.csv

In [ ]:
Ignore bellow This

In [ ]:
#vdo_canvas = generate_hand_keypoints_for_one_video(video_frames[0])
#len(vdo_canvas)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(vdo_canvas[30]) #canvas[:, :, [2, 1, 0]]
plt.axis('off')
plt.show()

In [ ]:
#pip install napari[all] napari_video
import napari
from napari_video.napari_video import VideoReaderNP

In [ ]:
vdos = []

In [ ]:
type(video_frames[0][0])

In [ ]:
vdos.append(VideoReaderNP('/kaggle/input/include-50/Animals/4. Bird/MVI_2987.MOV'))


In [ ]:
#figure ret val and vdos_keypoints, vdo_keypoints
def generate_hand_keypoints_for_one_video(video):
    vdo_canvas = []
    vdo_keypoints = []
    Num_of_frames = len(video)
    for f in range(Num_of_frames):
        oriImg = video[f]

        candidate, subset = body_estimation(oriImg)
        canvas = copy.deepcopy(oriImg)
        hands_list = util.handDetect(candidate, subset, oriImg)
        all_hand_peaks = get_all_hand_peaks(hands_list,oriImg)
        canvas = util.draw_handpose(canvas, all_hand_peaks)
        vdo_canvas.append(canvas)
        #vdos_keypoints.append(np.array(vdo_keypoints,dtype = 'float32'))
    return vdo_canvas

This function should take one vdo and return its hand keypoint, then append those keypoints to X. check Vee's paf codes too.

When it fails to detect hands, co ordinates are 0,0, we want that to be ignored. we dont want the model to be trained on that.

In [ ]:
vdo_canvas = generate_hand_keypoints_for_one_video(vdos[0])

In [ ]:
len(vdo_canvas)
#76X288X512X3
#frames x 

In [ ]:
labels1= ['Bird','Black','Car','Dog','Fall','Father','Good Morning', 'Red','Summer','White','loud','quiet','happy','long','short','big large', 'small little','hot', 'new','dry','good','Cow','Hat', 'T-Shirt', 'Shoes', 'Monday', 'Year', 'Time', 'Fan', 'Cell phone', 'Hello', 'Thank you', 'Window', 'Pen', 'Paint', 'Teacher', 'Priest', 'train ticket', 'Brother', 'Boy', ' Girl', ' Bank', 'I', 'it','you', 'Election', 'Death', 'Court', 'House', 'Store or Shop']

In [ ]:
lines = [24,15,16,13,12,18,17,18,11,19,15,18,18,15,16,17,18,15,18,21,18,18,14,17,16,12,8,12,12,10,14,14,11,12,12,11,14,14,16,17,18,16,18,12,18,12,9,20,20,18]

In [ ]:
len(labels1)

In [ ]:
vdos

In [ ]:
y = []

In [ ]:
for i in range(49):
    for j in range(1,lines[i]):
        y.append(i)

In [ ]:
len(y)

Dask Trials
https://blog.dask.org/2019/06/20/load-image-data

https://napari.org/stable/tutorials/processing/dask.html
https://github.com/dask/dask-examples/blob/main/applications/image-processing.ipynb

Tried:
1. glob returns an array of all filenames, imread takes glob like string and not the array.
iglob does return an iterator but it still isn't working. It says:
Could not detect your file type because it did not have an extension. Try specifying a loader class, e.g. Video(<generator object _iglob at 0x7f5ec4c5c1d0>)
To Do: Figure out how to specify a loader class.


2. Dask requirements.txt or environment.yml not yet run
I tried to use the binder environment (requires ur repo to be public, did that) butt it says:
package libgcc-ng-12.2.0-h65d4601_19 requires _libgcc_mutex 0.1 conda_forge, but none of the providers can be installed

To Do: Install all the requirements and run

3. Read vdos using napari and use dask array.. how to increase dask array...
TO DO: checkout AI4bharat's work on include.

In [ ]:
filename_pattern1 = os.path.join('/kaggle/input/include/Animals_1of2/Animals/4. Bird','MVI*.MOV')

In [ ]:
filename_pattern1

In [ ]:
tiled_astronaut_images = dask_image.imread.imread(filename_pattern1)

In [ ]:
print('\nNamed with wildcard *:')
for name in glob.iglob('/kaggle/input/include-50/Animals/4. Bird/*.M*'):
    print(name)

In [ ]:
filename_pattern1 = os.path.join('/kaggle/input/include-50/Animals/4. Bird','MVI_2987.MOV')
one_vdo = dask_image.imread.imread(filename_pattern1)
#dask_image.imread.imread('/kaggle/input/include-50/Animals/4. Bird/MVI_2987.MOV')

In [ ]:
one_vdo

In [ ]:
type(one_vdo)

In [ ]:
filenames = glob.glob('/kaggle/input/include-50/Animals/4. Bird/*.MOV')


In [ ]:
lazy_arrays = [dask.delayed(imageio.imread)(fn) for fn in filenames]
#lazy_arrays = [da.from_delayed(x, shape=sample.shape, dtype=sample.dtype)
               #for x in lazy_arrays]

In [ ]:
filenames

In [ ]:
lazy_arrays

In [ ]:
lazy_arrays[0].compute()

In [ ]:
import dask_image
x = dask_image.imread.imread('/kaggle/input/include-50/Animals/4. Bird/*.MOV')